In [ ]:
from scipy.io import loadmat
import numpy as np

#SETUP
X = loadmat("RawData.mat")['X']
y = loadmat("RawData.mat")['y']
Xones = np.ones((len(X),1))

#Single threshold for classification
thresh = 1500
for i in range(len(X)):
    if(y[i] <= thresh):
        y[i] = 0
    if(y[i] > thresh):
        y[i] = 1

#print(np.mean(y))

#Optional: Eliminate high values
#for i in range(110):
#    X = np.delete(X, np.argmax(y), 0)
#    y = np.delete(y, np.argmax(y), 0)
#    X = np.delete(X, np.argmin(y), 0)
#    y = np.delete(y, np.argmin(y), 0)

TwoNormCol = np.zeros((len(X.T),1))

#Remove keyword columns (poorly treated data)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)

#Normalize columns to 2-norm
for i in range(len(X.T)):
    TwoNormCol[i] = np.sqrt(X[i,:]@X[i,:])
    X[i,:] = X[i,:]/TwoNormCol[i]
#print(TwoNormCol)
#print(X[0,:])
#print(y)
    
#Form subsets (indices, first group is full X)
Xsubs = np.array([[0,49],[0,5],[5,7],[7,9],[9,11],[11,17],[17,20],[20,26],[28,33],[33,37],[37,45],[45,49]])

#Create and run over 11 sets of 3604 entries from X and y,
#using 1 as a primary test set (when needed for lambda)
setArr = np.array([[0,int(len(X)/11)],[int(len(X)/11),int(2*len(X)/11)],[int(2*len(X)/11),int(3*len(X)/11)],\
                   [int(3*len(X)/11),int(4*len(X)/11)],[int(4*len(X)/11),int(5*len(X)/11)],\
                   [int(5*len(X)/11),int(6*len(X)/11)],[int(6*len(X)/11),int(7*len(X)/11)],\
                   [int(7*len(X)/11),int(8*len(X)/11)],[int(8*len(X)/11),int(9*len(X)/11)],\
                   [int(9*len(X)/11),int(10*len(X)/11)],[int(10*len(X)/11),int(len(X))]])
#print(setArr)

#Set up network node count
networkCount = 3
networkTest = np.linspace(2, 6, networkCount)
#print(networkCount)

#Error tally storage
errorOrig = np.zeros((110,1))


################################

#Only looping over full X matrix
Xset = 0

M = 2

errorOrig = np.zeros((12,1))
errorNew = np.zeros((12,1))

for Xset in range(len(Xsubs)):
    for i in range(11):
        print("X subset = ",Xset,", i = ",i)
        testTally = 0
        #Set up Training and testing sets
        for k in range(11):
            if i == k:
                XTest1 = X[setArr[i,0]:setArr[i,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                #print(XTest1)
                yTest1 = y[setArr[i,0]:setArr[i,1]]
                #print(yTest1)
            if k != i:
                if testTally == 0:
                    XTrain = X[setArr[k,0]:setArr[k,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                    yTrain = y[setArr[k,0]:setArr[k,1]]
                    testTally = 1
                else:
                    XTrain = np.concatenate((XTrain, X[setArr[k,0]:setArr[k,1],Xsubs[Xset,0]:Xsubs[Xset,1]]),\
                                            axis=0)
                    yTrain = np.concatenate((yTrain, y[setArr[k,0]:setArr[k,1]]), axis=0)

        XTest1 = np.hstack((np.ones((len(XTest1),1)), XTest1))
        XTrain = np.hstack((np.ones((len(XTrain),1)), XTrain))
                
        # Storage for current iteration
        #WNET = np.zeros((len(X.T),networkCount))
                
        # Loop through different counts
        #for index in range(networkCount):
            #M = networkTest[index]
                
        V = np.random.randn(M+1, 1); 
        W = np.random.randn(len(XTrain.T), M);
                
        #Step and epoch count
        alpha = 0.5
        L = 10
                
        def logsig(_x):
            return 1/(1+np.exp(-_x))
                    
        for epoch in range(L):
            ind = np.random.permutation(len(XTrain))
            for iii in ind:
                # Forward propagation
                H = logsig(np.hstack((np.ones((1,1)), XTrain[[iii],:]@W)))
                #print(np.hstack((np.ones((1,1)), XTrain[[iii],:]@W)))
                Yhat = logsig(H@V)
                    
                # Back propagation
                delta = (Yhat-yTrain[[iii]])*Yhat*(1-Yhat)
                Vnew = V-alpha*H.T@delta
                gamma = delta@V[1:,:].T*H[:,1:]*(1-H[:,1:])
                Wnew = W - alpha*XTrain[[iii],:].T@gamma
                #print(alpha*XTrain[[iii],:].T@gamma)
                V = Vnew
                W = Wnew
            print(epoch)
            
            
        H = logsig(np.hstack((np.ones((len(XTrain),1)), XTrain@W)))
        y_net1 = logsig(H@V)
        
        errorOrigTemp = 0
        for index in range(len(XTrain)):
            if (yTrain[index] > 0.5 and y_net1[index] < 0.5) or (yTrain[index] < 0.5 and y_net1[index] > 0.5):
                errorOrigTemp = errorOrigTemp + 1
        errorOrigTemp = errorOrigTemp/len(XTrain)
        print("Mean error with orig set: ",errorOrigTemp)
        
        H = logsig(np.hstack((np.ones((len(XTest1),1)), XTest1@W)))
        y_net1 = logsig(H@V)
        
        errorOrig[Xset] = errorOrig[Xset] + errorOrigTemp
        
        errorOrigTemp = 0
        for index in range(len(XTest1)):
            if (yTest1[index] > 0.5 and y_net1[index] < 0.5) or (yTest1[index] < 0.5 and y_net1[index] > 0.5):
                errorOrigTemp = errorOrigTemp + 1
        errorOrigTemp = errorOrigTemp/len(yTest1)
        print("Mean error with test set: ",errorOrigTemp)
        
        errorNew[Xset] = errorNew[Xset] + errorOrigTemp

            
            
GlobalMeanOrigError = np.zeros((11,1))
GlobalMeanNewError = np.zeros((11,1))
for index in range(11):
    GlobalMeanOrigError[index] = errorOrig[index]/11
    GlobalMeanNewError[index] = errorNew[index]/11
    print("Mean Error for training set, X subset ",(index+1)," = ",GlobalMeanOrigError[index])
    print("Mean Error for test set, X subset ",(index+1)," = ",GlobalMeanNewError[index])
            
            

X subset =  0 , i =  0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:117: RuntimeWarning: overflow encountered in exp


0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.5443118756936737
Mean error with test set:  0.4961154273029967
X subset =  0 , i =  1
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.4514428412874584
Mean error with test set:  0.5463374028856826
X subset =  0 , i =  2
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.465788013318535
Mean error with test set:  0.40288568257491675
X subset =  0 , i =  3
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.463984461709212
Mean error with test set:  0.41509433962264153
X subset =  0 , i =  4
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.4596281908990011
Mean error with test set:  0.46448390677025525
X subset =  0 , i =  5
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.4554661487236404
Mean error with test set:  0.5061043285238623
X subset =  0 , i =  6
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.45493895671476137
Mean error with test set:  0.4961154273029967
X subset =  0 , i =  7
0
1
2
3
4
5
6
7
8
9
Mean error with orig set:  0.4583